In [1]:
from docx import Document
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.shared import RGBColor
from docx.shared import Cm
from docx.enum.table import WD_ROW_HEIGHT_RULE
import pandas as pd
import os

In [2]:
def drop_unnamed_cols(df):
    
    unnamed_cols = [col for col in df.columns
                   if col.startswith('Unnamed:')]
    
    df.drop(unnamed_cols, axis=1, inplace=True)

In [3]:
def read_data(sheetname, file, 
              path = 'original_data/consulta_publica/'):
    
    fpath = os.path.join(path, file)
    
    df = pd.read_excel(fpath, sheet_name = sheetname)
    drop_unnamed_cols(df)
    
    return df

In [4]:
def orange_cell(cell):
    
    # Set a cell background (shading) color to RGB D9D9D9.
    shading_elm = parse_xml(r'<w:shd {} w:fill="#FF7F50"/>'.format(nsdecls('w')))
    cell._tc.get_or_add_tcPr().append(shading_elm)

In [5]:
def write_white(cell, text):
    
    if text is None:
        text = ''
    if type(text) is not str:
        text = str(text)
    
    text = cell.paragraphs[0].add_run(text)
    white = RGBColor(255,255,255)
    text.font.color.rgb = white

In [6]:
def white_on_orange(cell, text):
    
    orange_cell(cell)
    write_white(cell, text)

In [7]:
def white_row(table, row_num, max_cell):
    
    merge_cells(table, row_num, max_cell)
    row = table.rows[row_num]
    row.height_rule = WD_ROW_HEIGHT_RULE.EXACTLY
    row.height = Cm(.3)

In [8]:
def add_table(doc, num_rows, num_cols):
    
    table = doc.add_table(num_rows,num_cols)
    table.style = 'Table Grid'
    
    return table

In [9]:
def merge_cells(table, row, max_cell, start_cell=0):
    
    merge_anchor = table.cell(row, start_cell)
    for i in range(start_cell+1, max_cell+1):
        merge_anchor.merge(table.cell(row, i))
    
    return merge_anchor

In [10]:
def write_cell(table, row, col, text):
    
    if text is None:
        text = ''
    
    if type(text) is not str:
        text = str(text)
    
    table.cell(row, col).text = text

In [11]:
def header(doc, first_row):
        
    table = add_table(doc, 2,2)
    white_on_orange(table.cell(0,0), first_row['Nº Meta'])
    write_cell(table, 0, 1, first_row['Descrição da Meta Municipal 2030'])
    white_row(table, 1, 1)

In [12]:
def vinculacao_pdm(doc, df):
    
    if len(df)<1:
        return
    
    if df.iloc[0]['Meta PdM (Número)']=='sem vinculação':
        return
    
    #uma para cada vinculo, uma para header e outra para espaço
    num_rows = len(df)+2
    table = add_table(doc, num_rows, 2)
    
    header = merge_cells(table, 0, 1)
    white_on_orange(header, 'Vinculação com o Programa de Metas 2021-2024')
    
    for i, row in df.iterrows():
        
        write_cell(table, i+1,0, row['Meta PdM (Número)'])
        write_cell(table, i+1,1, row['Meta PdM (Descrição)'])

    white_row(table, num_rows-1, 1)

In [13]:
def vinculacao_ppa(doc, df):
    
    #uma para cada vinculo, uma para header e outra para espaço
    num_rows = len(df)+2
    table = add_table(doc, num_rows, 4)
    
    header = merge_cells(table, 0, 3)
    white_on_orange(header, 'Vinculação com o Plano Plurianual 2022-2025')
    
    for i, row in df.iterrows():
        
        programa = merge_cells(table, i+1, 1)
        write_cell(table, i+1,0, row['Programa PPA (Número)'])
        write_cell(table, i+1,2, row['Programa PPA (Descrição)'])
        write_cell(table, i+1,3, row['Objetivo PPA'])
    
    white_row(table, num_rows-1, 1)

In [14]:
def indicadores_priorizados(doc, df):
    
    if len(df) < 1:
        return
    #uma para cada item, duas para header e outra para espaço
    num_rows = len(df)+3
    table = add_table(doc, num_rows, 4)
    
    header = merge_cells(table, 0, 3)
    white_on_orange(header, 'Indicadores da Agenda Municipal 2030 priorizados')
    
    white_on_orange(table.cell(1,0), 'Número')
    white_on_orange(table.cell(1,1), 'Redação')
    white_on_orange(table.cell(1,2), 'Meta 2030')
    white_on_orange(table.cell(1,3), 'Previsão 2021-2024')

    
    for i, row in df.iterrows():
        
        row_num = i+2
        write_cell(table, row_num,0, row['Nº do Indicador'])
        write_cell(table, row_num,1, row['Nome do Indicador'])
        write_cell(table, row_num,2, row['Meta numérica 2030 para o Indicador'])
        write_cell(table, row_num,3, row['Previsão de desempenho 2024 para o Indicador\n(TEXTO A SER VALIDADO)'])

    white_row(table, num_rows-1, 3)

In [15]:
def acoes(doc, df):

    #uma para cada item, duas para header
    num_rows = len(df)+1
    table = add_table(doc, num_rows, 4)
    
    header = merge_cells(table, 0, 1)
    white_on_orange(header, 'Ação')
    white_on_orange(table.cell(0,2), 'Orgão Responsável')
    white_on_orange(table.cell(0,3), 'Marco de Atingimento')
    
    for i, row in df.iterrows():
        
        row_num = i+1
        
        write_cell(table, row_num,0, row['Letra da Ação'])
        write_cell(table, row_num,1, row['Descrição da Ação\n(TEXTO A SER VALIDADO)'])
        write_cell(table, row_num,2, row['Órgão Responsável'])
        write_cell(table, row_num,3, row['marco'])


In [16]:
def filtrar_df(df, meta):
    
    if 'Meta 2030 (Número)' in df:
        col_meta = 'Meta 2030 (Número)'
    else:
        col_meta = 'Nº Meta'
    
    filtro = df[col_meta] == meta
    
    df = df[filtro].copy().reset_index(drop=True)
    
    return df

In [17]:
def open_sheet(sheet_name, meta, file = None):
    
    sheets = {'pdm' : 'PdM',
          'ppa' : 'PPA',
          'indicadores' : 'Indicadores_priorizados',
          'acoes' : 'Sheet1'}
    
    df = read_data(sheets[sheet_name], file=file)
    
    filtered = filtrar_df(df, meta)
    
    return filtered

In [18]:
doc = Document()

In [19]:
def sort_meta(metas):
    result = []
    
    for meta in metas:
        try:
            result.append(float(meta))
        except ValueError:
            inteiro, letra = meta.split('.')
            letra = letra.lower()
            inteiro = int(inteiro)
            letra_num = ord(letra)
            
            result.append(float(f'{inteiro}.{9}{letra_num}'))
            
    return pd.Series(result)

In [20]:
def sort_acao(acoes):
    result = []
    
    for acao in acoes:
        letra = acao.split('-')[-1]
        letra = letra.strip()
        letra = letra.lower()
        letra_num = ord(letra)-96
        result.append(letra_num)
    return pd.Series(result)

In [21]:
metas = read_data('Ações_propostas', file = 'producao.xlsx')
metas = metas.sort_values(by='Nº Meta', key = sort_meta).reset_index(drop=True)
metas = metas['Nº Meta'].unique()

for meta in metas:
    
    df_acoes = open_sheet('acoes', meta, file = 'acoes_com_marcos_VF.xlsx')
    df_acoes = df_acoes.sort_values(by='Letra da Ação', key=sort_acao).reset_index(drop=True)
    df_pdm = open_sheet('pdm', meta, file = 'producao.xlsx')
    df_ppa = open_sheet('ppa', meta, file = 'producao.xlsx')
    df_indicadores = open_sheet('indicadores', meta, file = 'producao.xlsx')
    
    
    header(doc, df_acoes.iloc[0])
    vinculacao_pdm(doc, df_pdm)
    vinculacao_ppa(doc, df_ppa)
    indicadores_priorizados(doc, df_indicadores)
    acoes(doc, df_acoes)
    doc.add_page_break()

In [22]:
file_name = 'relatorio_consulta_publica_final.docx'
if os.path.exists(file_name):
    os.remove(file_name)
assert not os.path.exists(file_name)

In [23]:
doc.save(file_name)

In [37]:
df = read_data('Ações_propostas', file = 'producao.xlsx')

In [43]:
df = df.sort_values(by='Órgão Responsável', key = lambda series: pd.Series([item.lower() for item in series]))

In [44]:
for i, row in df.iterrows():
    print(i)
    print(row['Órgão Responsável'])

143
CGM
164
CGM
142
CGM
179
CGM
178
CGM
177
CGM
176
CGM
166
CGM
167
CGM
165
CGM
161
CGM
160
CGM
159
CGM
158
CGM
144
CGM
163
CGM
162
CGM
235
PGM
236
PGM
239
PGM 
358
SEHAB
640
SEHAB
357
SEHAB
513
SEHAB
514
SEHAB
515
SEHAB
429
SEME
428
SEME
234
SF
238
SF
237
SF
188
SGM
189
SGM
253
SGM
20
SGM
610
SGM
223
SGM
264
SGM
252
SGM
186
SGM
327
SGM
183
SGM
255
SGM
256
SGM
257
SGM
200
SGM
609
SGM
185
SGM
224
SGM
184
SGM
254
SGM
209
SGM
251
SGM
210
SGM
205
SGM
187
SGM
263
SGM
568
SGM
262
SGM
203
SGM
260
SGM
208
SGM
207
SGM
206
SGM
204
SGM
211
SGM
250
SGM
249
SGM
401
SGM
261
SGM
217
SGM
607
SGM; SIURB
368
SGM; SMADS; SMS; SME; SMDHC
198
SGM; SMDHC; SMDET
611
SGM; SVMA
608
SGM; SVMA
499
SIURB
500
SIURB
502
SIURB
328
SMADS
392
SMADS
391
SMADS
329
SMADS
394
SMADS
372
SMADS
330
SMADS
403
SMADS
311
SMADS
310
SMADS
307
SMADS
397
SMADS
396
SMADS
395
SMADS
331
SMADS
367
SMADS
333
SMADS
390
SMADS
389
SMADS
336
SMADS
369
SMADS
338
SMADS
393
SMADS
373
SMADS
388
SMADS
348
SMADS
332
SMADS
366
SMADS
308
SMADS; SMD

In [24]:
#pd.read_excel('original_data/consulta_publica/acoes_com_marcos_VF.xlsx').keys()